In [ ]:
type = "cron"

In [ ]:
import pandas as pd
import os
from pathlib import Path

In [ ]:
dir = Path("C:/Users/andre/Documents/Talon persisted files")
files = os.listdir(dir)
files = [f for f in files if f.startswith(type)]
print(files)

In [ ]:
df = pd.DataFrame()
for file in files:
    file = dir / file
    temp = pd.read_json(file, lines=True)
    df = pd.concat([df, temp])
df["date"] = pd.to_datetime(df["time"], unit="s").dt.strftime("%Y-%m-%d")
df.shape

In [ ]:
def calculate_cron(df):
    df.loc["Total",:]= df[["count", "sum", "window_size"]].sum(axis=0, numeric_only=True)
    df = df.replace(pd.NA, "-")
    df = df.groupby(["date", "application"])[["count", "sum", "window_size"]].mean()
    # Average deviation in milliseconds
    df["avg deviation (ms)"] = df["sum"] / df["count"]
    # Count per second
    df["count / s"] = df["count"] / df["window_size"]
    df.drop(["count", "sum", "window_size"], axis=1, inplace=True)
    return df.round(1)

def calculate_phrase(df):
    df.loc["Total",:]= df[["compile_ms", "total_ms"]].mean(axis=0, numeric_only=True)
    df = df.replace(pd.NA, "-")
    df = df.groupby(["date", "application"])[["compile_ms", "total_ms"]].mean()
    return df.round(1)

def calculate(df):
    if type == "cron":
        return calculate_cron(df)
    if type == "phrase":
        return calculate_phrase(df)
    return 

calculate(df.copy())